# No-glue-code

## Model

In [1]:
# The statistical inference frame-work we will use
using Turing
using AdvancedHMC
using LogDensityProblems
using LogDensityProblemsAD
using DynamicPPL
using ForwardDiff
# Some data management libs.
using CSV
using NPZ
using YAML
#Plotting
using Plots
# Some Lin. Alg.
using LinearAlgebra
using Interpolations

[ Info: Precompiling Turing [fce5fe82-541a-59a6-adf8-730c64b5f9a0]


In [2]:
fs8_zs = [0.38, 0.51, 0.61, 1.48, 0.44, 0.6, 0.73, 0.6, 0.86, 0.067, 1.4]
fs8_data = [0.49749, 0.457523, 0.436148, 0.462, 0.413, 0.39, 0.437, 0.55, 0.4, 0.423, 0.482]
fs8_cov = [0.00203355 0.000811829 0.000264615 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
           0.000811829 0.00142289 0.000662824 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 
           0.000264615 0.000662824 0.00118576 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
           0.0 0.0 0.0 0.002025 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
           0.0 0.0 0.0 0.0 0.0064 0.00257 0.0 0.0 0.0 0.0 0.0;
           0.0 0.0 0.0 0.0 0.00257 0.003969 0.00254 0.0 0.0 0.0 0.0;
           0.0 0.0 0.0 0.0 0.0 0.00254 0.005184 0.0 0.0 0.0 0.0;
           0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0144 0.0 0.0 0.0;
           0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0121 0.0 0.0; 
           0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.003025 0.0;
           0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.013456000000000001];

In [3]:
function make_fs8(Ωm, σ8; Ωr=8.24*10^-5)
    # ODE solution for growth factor
    x_Dz = LinRange(0, log(1+1100), 300)
    dx_Dz = x_Dz[2]-x_Dz[1]
    z_Dz = @.(exp(x_Dz) - 1)
    a_Dz = @.(1/(1+z_Dz))
    aa = reverse(a_Dz)
    e =  @.(sqrt.(abs(Ωm)*(1+z_Dz)^3+Ωr*(1+z_Dz)^4+(1-Ωm-Ωr)))
    ee = reverse(e)

    dd = zeros(typeof(Ωm), 300)
    yy = zeros(typeof(Ωm), 300)
    dd[1] = aa[1]
    yy[1] = aa[1]^3*ee[end]

    for i in 1:(300-1)
        A0 = -1.5 * Ωm / (aa[i]*ee[i])
        B0 = -1. / (aa[i]^2*ee[i])
        A1 = -1.5 * Ωm / (aa[i+1]*ee[i+1])
        B1 = -1. / (aa[i+1]^2*ee[i+1])
        yy[i+1] = (1+0.5*dx_Dz^2*A0*B0)*yy[i] + 0.5*(A0+A1)*dx_Dz*dd[i]
        dd[i+1] = 0.5*(B0+B1)*dx_Dz*yy[i] + (1+0.5*dx_Dz^2*A0*B0)*dd[i]
    end

    y = reverse(yy)
    d = reverse(dd)

    Dzi = linear_interpolation(z_Dz, d./d[1], extrapolation_bc=Line())
    fs8zi = linear_interpolation(z_Dz, -σ8 .* y./ (a_Dz.^2 .*e.*d[1]),
                                 extrapolation_bc=Line())
    return fs8zi
end

make_fs8 (generic function with 1 method)

In [4]:
@model function model(data; cov = fs8_cov) 
    # Define priors
    #KiDS priors
    Ωm ~ Uniform(0.1, 0.9)
    σ8 ~ Uniform(0.4, 1.2)
    fs8_itp = make_fs8(Ωm, σ8)
    theory = fs8_itp(fs8_zs)
    data ~ MvNormal(theory, cov)
end;

In [5]:
stat_model = model(fs8_data)

Model{typeof(model), (:data, :cov), (:cov,), (), Tuple{Vector{Float64}, Matrix{Float64}}, Tuple{Matrix{Float64}}, DefaultContext}(model, (data = [0.49749, 0.457523, 0.436148, 0.462, 0.413, 0.39, 0.437, 0.55, 0.4, 0.423, 0.482], cov = [0.00203355 0.000811829 … 0.0 0.0; 0.000811829 0.00142289 … 0.0 0.0; … ; 0.0 0.0 … 0.003025 0.0; 0.0 0.0 … 0.0 0.013456000000000001]), (cov = [0.00203355 0.000811829 … 0.0 0.0; 0.000811829 0.00142289 … 0.0 0.0; … ; 0.0 0.0 … 0.003025 0.0; 0.0 0.0 … 0.0 0.013456000000000001],), DefaultContext())

In [6]:
adaptation = 1000
TAP = 0.65
alg = Turing.NUTS(adaptation, TAP)

Turing.Inference.NUTS{Turing.Essential.ForwardDiffAD{0}, (), DiagEuclideanMetric}(1000, 0.65, 10, 1000.0, 0.0)

## Getting MAP and Hessian

In [7]:
using Optim

function Xi2(params; model=stat_model)
    Ωm, σ8 = params
    return loglikelihood(model, (Ωm=Ωm, σ8=σ8))
end;
    
function maximum_a_posteriori(model, lower_bound, upper_bound)
    start_value = (upper_bound .+ lower_bound) ./ 2 
    opt = optimize((v)->-Xi2(v), lower_bound, upper_bound, start_value, Fminbox())
    return Optim.minimizer(opt)
end

maximum_a_posteriori (generic function with 1 method)

In [8]:
MAP = maximum_a_posteriori(stat_model, [0.2, 0.4], [0.6, 1.2])

2-element Vector{Float64}:
 0.21256856797862178
 0.8763540154601552

In [9]:
# Get the Hessian
hess = ForwardDiff.hessian(Xi2, MAP)
inv_hess = inv(hess);

In [10]:
# Turn the Hessian into more of a covariance Matrix
w, v = eigen(inv_hess)
hess_cov = v * (diagm(abs.(w)) * v')
hess_cov = tril(hess_cov) + triu(hess_cov', 1)
hess_cov = Hermitian(hess_cov)
hess_cov = convert(Matrix{Float64}, hess_cov)

2×2 Matrix{Float64}:
  0.00383693  -0.00328434
 -0.00328434   0.0042819

## Sampling

In [11]:
spl = Sampler(alg, stat_model)

Sampler{Turing.Inference.NUTS{Turing.Essential.ForwardDiffAD{0}, (), DiagEuclideanMetric}}(Turing.Inference.NUTS{Turing.Essential.ForwardDiffAD{0}, (), DiagEuclideanMetric}(1000, 0.65, 10, 1000.0, 0.0), DynamicPPL.Selector(0x000005c4fc804c8d, :default, false))

In [12]:
context = stat_model.context
varinfo = DynamicPPL.VarInfo(stat_model, context)

TypedVarInfo{NamedTuple{(:Ωm, :σ8), Tuple{DynamicPPL.Metadata{Dict{VarName{:Ωm, Setfield.IdentityLens}, Int64}, Vector{Uniform{Float64}}, Vector{VarName{:Ωm, Setfield.IdentityLens}}, Vector{Float64}, Vector{Set{DynamicPPL.Selector}}}, DynamicPPL.Metadata{Dict{VarName{:σ8, Setfield.IdentityLens}, Int64}, Vector{Uniform{Float64}}, Vector{VarName{:σ8, Setfield.IdentityLens}}, Vector{Float64}, Vector{Set{DynamicPPL.Selector}}}}}, Float64}((Ωm = DynamicPPL.Metadata{Dict{VarName{:Ωm, Setfield.IdentityLens}, Int64}, Vector{Uniform{Float64}}, Vector{VarName{:Ωm, Setfield.IdentityLens}}, Vector{Float64}, Vector{Set{DynamicPPL.Selector}}}(Dict(Ωm => 1), [Ωm], UnitRange{Int64}[1:1], [0.24189169385043288], Uniform{Float64}[Uniform{Float64}(a=0.1, b=0.9)], Set{DynamicPPL.Selector}[Set()], [0], Dict{String, BitVector}("del" => [0], "trans" => [0])), σ8 = DynamicPPL.Metadata{Dict{VarName{:σ8, Setfield.IdentityLens}, Int64}, Vector{Uniform{Float64}}, Vector{VarName{:σ8, Setfield.IdentityLens}}, Vector

In [14]:
ℓ = LogDensityProblemsAD.ADgradient(DynamicPPL.LogDensityFunction(varinfo, stat_model, context))
lπ = Base.Fix1(LogDensityProblems.logdensity, ℓ)
∂lπ∂θ(x) = LogDensityProblems.logdensity_and_gradient(ℓ, x)
metric = DenseEuclideanMetric(hess_cov)
hamiltonian = AdvancedHMC.Hamiltonian(metric, lπ, ∂lπ∂θ)

Hamiltonian(metric=DenseEuclideanMetric(diag=[0.003836928914103148, 0.00 ...]), kinetic=GaussianKinetic())

In [15]:
# Set the number of samples to draw and warmup iterations
n_samples, n_adapts = 2_000, 1_000
initial_ϵ = find_good_stepsize(hamiltonian, MAP)
integrator = Leapfrog(initial_ϵ)

# Define an HMC sampler, with the following components
#   - multinomial sampling scheme,
#   - generalised No-U-Turn criteria, and
#   - windowed adaption for step-size and diagonal mass matrix
proposal = AdvancedHMC.NUTS{MultinomialTS, GeneralisedNoUTurn}(integrator)
adaptor = StanHMCAdaptor(MassMatrixAdaptor(metric), StepSizeAdaptor(0.8, integrator))

# Run the sampler to draw samples from the specified Gaussian, where
#   - `samples` will store the samples
#   - `stats` will store diagnostic statistics for each sample
samples, stats = sample(hamiltonian, proposal, MAP, n_samples, adaptor, n_adapts; progress=true)

┌ Warning: ProgressMeter by default refresh meters with additional information in IJulia via `IJulia.clear_output`, which clears all outputs in the cell. 
│  - To prevent this behaviour, do `ProgressMeter.ijulia_behavior(:append)`. 
│  - To disable this warning message, do `ProgressMeter.ijulia_behavior(:clear)`.
└ @ ProgressMeter ~/.julia/packages/ProgressMeter/sN2xr/src/ProgressMeter.jl:618
Sampling 100%|███████████████████████████████| Time: 0:00:01
  iterations:                    2000
  n_steps:                       3
  is_accept:                     true
  acceptance_rate:               1.0
  log_density:                   17.729748960723036
  hamiltonian_energy:            -17.289392697793396
  hamiltonian_energy_error:      -0.051060919913108904
  max_hamiltonian_energy_error:  -0.06881331813539404
  tree_depth:                    2
  numerical_error:               false
  step_size:                     0.6487150772760925
  nom_step_size:                 0.6487150772760925
  i

([[0.24217072449163063, 0.8815235005982114], [0.24217072449163063, 0.8815235005982114], [0.1964040900666372, 0.8682642940757254], [0.23220726706788417, 0.8594888321891413], [0.23220726706788417, 0.8594888321891413], [0.30600748508356646, 0.8187262839439398], [0.24638113811106768, 0.8264874921491668], [0.17930813221283726, 0.9012387514591719], [0.257747112897827, 0.8659528543802886], [0.2219106258796527, 0.872052499881699]  …  [0.2155585668611011, 0.9387916846186595], [0.30467165855486844, 0.8016599731719438], [0.19238795508040302, 0.8454240746568259], [0.23402240097427798, 0.7968218235978053], [0.2694905067075098, 0.9217373336532368], [0.1882021323995838, 0.8599530394625028], [0.23797097860528293, 0.8813057277500905], [0.2943973036254594, 0.8006200200955638], [0.15372119929153283, 0.9463410337224535], [0.2267280941717041, 0.8903478326313119]], NamedTuple[(n_steps = 7, is_accept = true, acceptance_rate = 0.4004465309788565, log_density = 17.597878418351858, hamiltonian_energy = -17.5843